In [1]:
import pandas as pd
import numpy as np
import spacy
import preprocessing
from csv import reader 
import tfidf
import scipy

pd.set_option('max_colwidth', 5000)

df = pd.read_csv('DB/Original_data.csv', index_col=[0])
df = df.sample(frac=0.2)


In [2]:
#Check if there are null values 
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

In [3]:
#create column with answers aggregate by title of the questions
df2 = df 
func = lambda x: "\n".join(x)
df2 = df2.groupby('id')["answers"].agg([("answers",func)])



In [4]:
# concat answers for each post beacause there are a record with question duplicated for each answer to that question 


grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg("sum","score")

grouped_df = pd.DataFrame(grouped)
grouped_df = pd.merge(grouped_df, df2, left_on='id', right_on='id', how='left')
grouped_df



,id,title,body,tags,score,answers
0,742,Class views in Django,"<p><a href=""http://www.djangoproject.com/"" rel=""noreferrer"">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\n\n<p>For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc...</p>\n\n<p>One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea? </p>",python|django|views|oop,18,"<p>I needed to use class based views, but I wanted to be able to use the full name of the class in my URLconf without always having to instantiate the view class before using it. What helped me was a surprisingly simple metaclass:</p>\n\n<pre><code>class CallableViewClass(type):\n def __call__(cls, *args, **kwargs):\n if args and isinstance(args[0], HttpRequest):\n instance = super(CallableViewClass, cls).__call__()\n return instance.__call__(*args, **kwargs)\n else:\n instance = super(CallableViewClass, cls).__call__(*args, **kwargs)\n return instance\n\n\nclass View(object):\n __metaclass__ = CallableViewClass\n\n def __call__(self, request, *args, **kwargs):\n if hasattr(self, request.method):\n handler = getattr(self, request.method)\n if hasattr(handler, '__call__'):\n return handler(request, *args, **kwargs)\n return HttpResponseBadRequest('Method Not Allowed', status=405)\n</code></pre>\n\n<p>I can now both instantiate view classes and use the instances as view functions, OR I can simply point my URLconf to my class and have the metaclass instantiate (and call) the view class for me. This works by checking the first argument to <code>__call__</code> – if it's a <code>HttpRequest</code>, it must be an actual HTTP request because it would be nonsense to attept to instantiate a view class with an <code>HttpRequest</code> instance.</p>\n\n<pre><code>class MyView(View):\n def __init__(self, arg=None):\n self.arg = arg\n def GET(request):\n return HttpResponse(self.arg or 'no args provided')\n\n@login_required\nclass MyOtherView(View):\n def POST(request):\n pass\n\n# And all the following work as expected.\nurlpatterns = patterns(''\n url(r'^myview1$', 'myapp.views.MyView', name='myview1'),\n url(r'^myview2$', myapp.views.MyView, name='myview2'),\n url(r'^myview3$', myapp.views.MyView('foobar'), name='myview3'),\n url(r'^myotherview$', 'myapp.views.MyOtherView', name='otherview'),\n)\n</code></pre>\n\n<p>(I posted a snippet for this at <a href=""http://djangosnippets.org/snippets/2041/"" rel=""noreferrer"">http://djangosnippets.org/snippets/2041/</a>)</p>\n<p>Unless you want to do something a little complex, using the generic views are the way to go. They are far more powerful than their name implies, and if you are just displaying model data generic views will do the job.</p>\n<p>You can always create a class, override the <em><code>__call__</code></em> function and then point the URL file to an instance of the class. You can take a look at the <a href=""http://code.djangoproject.com/browser/django/trunk/django/contrib/formtools/wizard.py"" rel=""nofollow noreferrer"">FormWizard</a> class to see how this is done.</p>"
1,773,How do I use itertools.groupby()?,"<p>I haven't been able to find an understandable explanation of how to actually use Python's <code>itertools.groupby()</code> function. What I'm trying to do is this:</p>\n<ul>\n<li>Take a list - in this case, the children of an objectified <code>lxml</code> element</li>\n<li>Divide it into groups based on some criteria</li>\n<li>Then later iterate over each of these groups separately.</li>\n</ul>\n<p>I've reviewed <a href=""https://docs.python.org/3/library/itertools.html#itertools.groupby"" rel=""noreferrer"">the do

In [5]:
#Preprocessing Part
#Answers
#Removing tags


answers = grouped_df["answers"]
preprocessing.remove_tags(answers)



#Clearing text 
answers_processed = answers.apply(lambda x: preprocessing.clear_text(x))


In [6]:
answers_processed.head()
answers_processed.isna().sum() 

0

In [7]:
#Questions
#Merge title with body 
questions = grouped_df["title"] + grouped_df["body"]
preprocessing.remove_tags(questions)
questions

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Class views in Django  view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.    For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc...    One way would be to point views to class methods a

In [8]:
#Clearing text 
questions_processed = questions.apply(lambda x: preprocessing.clear_text(x))
questions_processed

0                                                                                                                                                                                                                                               class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea 
1                                                                                                                                                                                                                                                                             use itertools groupby able find understandable explanation actually use python function trying take list case childr

In [9]:
#Create a column to add in db with question title, question body and all the answers processed, to train later the W2V embeddings
post_corpus = questions_processed + answers_processed
grouped_df.drop("answers", axis=1, inplace=True)
grouped_df.drop("body", axis=1, inplace=True)
grouped_df["post_corpus"] = post_corpus
grouped_df["questions"] = questions
grouped_df

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,python|django|views|oop,18,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance unless want something little complex using generic views way go far powerful name implies displaying model data generic views job always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?"
1,773,How do I use itertools.groupby()?,python|itertools,2,use itertools groupby able find understandable explanation actually use python function trying take list case children objectified element divide groups based criteria later iterate groups separately reviewed trouble trying apply beyond simple list numbers use another technique using pointers good prerequisite reading would also appreciated one useful example came across may helpful sample input sample output,"How do I use itertools.groupby ? I haven't been able to find an understandable explanation of how to actually use Python's function. What I'm trying to do is this: Take a list - in this case, the children of an objectified element Divide it into groups based on some criteria Then later iterate over each of these groups separately. I've reviewed , but I've had trouble trying to apply them beyond a simple list of numbers. So, how do I use of ? Is there another technique I should be using? Pointers to good &quot;prerequisite&quot; reading would also be appreciated."
2,2933,Create a directly-executable cross-platform GUI app using Python,python|user-interface|deployment|tkinter|release-management,6,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line sure best way deploying ruby gui apps windows write short launcher c calls main script compiles executable application executable three things could first thing find gui designer launch code standalone applications like exe files use version matdeck believe allows convert code standalone applications need install software every pc going run program second option partially bypassing problem launch gui web page would give compatibility os utilize would need gui designer convert components web compatible format done used version matdeck would recommend md python designer time know turn guis websites using web assembly whereas visionary deck tried tested things likely software one use frequently work lot mathematics physics third option also kind bypassing problem tkinter ensure python ide plain old python run code la

In [10]:
#Filter out the tags, selecting only the 20 most common for better processing 
# Convert raw text data of tags into lists
grouped_df["tags"] = grouped_df["tags"].apply(lambda x: x.split('|'))   

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}

#i = 0 

for tags in grouped_df["tags"]:
    for tag in tags:
        #Remove tags python, python2.7 e python3 for further processing 
        if "python" not in tag :
            
            if tag not in tag_freq_dict:
                tag_freq_dict[tag] = 0
            else:
                tag_freq_dict[tag] += 1
        else:
            tags.remove(tag)
    '''
    #If a record has less than 3 tags eliminate it for better further processing (training of fastText)
    if len(tags) < 3:
        grouped_df.drop(i)
    i = i +1
    '''
grouped_df["tags"]


0                                             [django, views, oop]
1                                                      [itertools]
2        [user-interface, deployment, tkinter, release-management]
3                                                [apache, apache2]
4                                                      [c, matlab]
                                   ...                            
86992                             [function, return, return-value]
86993                                            [pyside6, qchart]
86994                                                           []
86995                           [regex, web-scraping, re, findall]
86996                  [if-statement, dutch-national-flag-problem]
Name: tags, Length: 86997, dtype: object

In [11]:
#Selecting the most common number of tags in our database 
import heapq
most_common_tags = heapq.nlargest(30, tag_freq_dict, key=tag_freq_dict.get)
most_common_tags

['dataframe',
 'numpy',
 'pandas',
 'dictionary',
 'list',
 'django',
 'matplotlib',
 'beautifulsoup',
 'flask',
 'csv',
 'tkinter',
 'keras',
 'django-models',
 'javascript',
 'web-scraping',
 'for-loop',
 'scipy',
 'scikit-learn',
 'datetime',
 'string',
 'pip',
 'loops',
 'selenium-webdriver',
 'pyspark',
 'sqlalchemy',
 'django-rest-framework',
 'machine-learning',
 'tuples',
 'java',
 'selenium']

In [12]:
#Select only the data with at least one of the most common tags 
final_indices = []
for i,tags in enumerate(grouped_df["tags"].values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>0:   # The minimum length for common tags should be 1
        final_indices.append(i)

final_data = grouped_df.iloc[final_indices]

final_data 

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,"[django, views, oop]",18,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance unless want something little complex using generic views way go far powerful name implies displaying model data generic views job always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?"
2,2933,Create a directly-executable cross-platform GUI app using Python,"[user-interface, deployment, tkinter, release-management]",6,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line sure best way deploying ruby gui apps windows write short launcher c calls main script compiles executable application executable three things could first thing find gui designer launch code standalone applications like exe files use version matdeck believe allows convert code standalone applications need install software every pc going run program second option partially bypassing problem launch gui web page would give compatibility os utilize would need gui designer convert components web compatible format done used version matdeck would recommend md python designer time know turn guis websites using web assembly whereas visionary deck tried tested things likely software one use frequently work lot mathematics physics third option also kind bypassing problem tkinter ensure python ide plain old python run code launch gui good solution maybe simplest class shortest best plan switch operating systems computers probably best bet,"Create a directly-executable cross-platform GUI app using Python Python works on multiple platforms and can be used for desktop and web applications, thus I conclude that there is some way to compile it into an executable for Mac, Windows and Linux. The problem being I have no idea where to start or how to write a GUI with it, can anybody shed some light on this and point me in the right direction please?"
6,21961,Date/time conversion using time.mktime seems wrong,[datetime],7,date time conversion using time mktime seems wrong return number seconds since epoch since giving time midnight epoch midnight result evenly divisible number seconds day short answer timezones epoch utc example ist utc function works utc,"Date/time conversion using time.mktime seems wrong should return the number of seconds since the epoch. Since I'm giving it a time at midnight and the epoch is at midnight, shouldn't the result be evenly di

In [13]:
#Create a column only for the processed title of the questions 
final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x))

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_20692/1199495892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x))


In [14]:
# Normalize numeric data for the scores
final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_20692/3151826978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())


In [15]:
# Combine the lists back into text data
final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))

final_data

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_20692/3494508092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))


,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.001921,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance unless want something little complex using generic views way go far powerful name implies displaying model data generic views job always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?",class views django
2,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.000289,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line sure best way deploying ruby gui apps windows write short launcher c calls main script compiles executable application executable three things could first thing find gui designer launch code standalone applications like exe files use version matdeck believe allows convert code standalone applications need install software every pc going run program second option partially bypassing problem launch gui web page would give compatibility os utilize would need gui designer convert components web compatible format done used version matdeck would recommend md python designer time know turn guis websites using web assembly whereas visionary deck tried tested things likely software one use frequently work lot mathematics physics third option also kind bypassing problem tkinter ensure python ide plain old python run code launch gui good solution maybe simplest class shortest best plan switch operating systems computers probably best bet,"Create a directly-executable cross-platform GUI app using Python Python works on multiple platforms and can be used for desktop and web applications, thus I conclude that there is some way to compile it into an executable for Mac, Windows and Linux. The problem being I have no idea where to start or how to write a GUI with it, can anybody shed some light on this and point me in the right direction please?",create directly executable cross platform gui app using python
6,21961,Date/time conversion using time.mktime seems wrong,datetime,0.000425,date time conversion using time mktime seems wrong return number seconds since epoch since giving time midnight epoch midnight result evenly divisible number seconds day short answer timezones epoch utc example ist utc function works utc,"Date/time conversion using time.mktime seems wrong should return the number of seconds since the epoch.

In [16]:
#Check if the final data has some null values 
final_data.isna().sum()

final_data = final_data.dropna()
#final_data = final_data[final_data['post_corpus'].notna()]
final_data = final_data[final_data['processed_title'].notna()]
final_data 

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.001921,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance unless want something little complex using generic views way go far powerful name implies displaying model data generic views job always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?",class views django
2,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.000289,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line sure best way deploying ruby gui apps windows write short launcher c calls main script compiles executable application executable three things could first thing find gui designer launch code standalone applications like exe files use version matdeck believe allows convert code standalone applications need install software every pc going run program second option partially bypassing problem launch gui web page would give compatibility os utilize would need gui designer convert components web compatible format done used version matdeck would recommend md python designer time know turn guis websites using web assembly whereas visionary deck tried tested things likely software one use frequently work lot mathematics physics third option also kind bypassing problem tkinter ensure python ide plain old python run code launch gui good solution maybe simplest class shortest best plan switch operating systems computers probably best bet,"Create a directly-executable cross-platform GUI app using Python Python works on multiple platforms and can be used for desktop and web applications, thus I conclude that there is some way to compile it into an executable for Mac, Windows and Linux. The problem being I have no idea where to start or how to write a GUI with it, can anybody shed some light on this and point me in the right direction please?",create directly executable cross platform gui app using python
6,21961,Date/time conversion using time.mktime seems wrong,datetime,0.000425,date time conversion using time mktime seems wrong return number seconds since epoch since giving time midnight epoch midnight result evenly divisible number seconds day short answer timezones epoch utc example ist utc function works utc,"Date/time conversion using time.mktime seems wrong should return the number of seconds since the epoch.

In [17]:
#Create matrix for tdfidf for further processing 
matrix = tfidf.create_matrix(final_data.post_corpus)
scipy.sparse.save_npz('DB/tfidf_stack_matrix.npz', matrix)

In [18]:
# Save the data
final_data.to_csv('DB/Preprocessed_data.csv', index=False)